In [1]:
!pip install textsearch
!pip install contractions
import nltk


#nltk.download('all')
nltk.download('punkt')
nltk.download('stopwords')

     |████████████████████████████████| 287 kB 9.1 MB/s 
     |████████████████████████████████| 106 kB 42.6 MB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [27]:
import numpy as np

In [20]:
import pandas as pd

df = pd.read_csv('https://github.com/dipanjanS/nlp_workshop_dhs18/raw/master/Unit%2010%20-%20Project%208%20-%20Movie%20Recommendations%20with%20Document%20Similarity/tmdb_5000_movies.csv.gz', compression='gzip')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [21]:
df = df[['title','tagline','overview']]

In [22]:
df['tagline'] = df['tagline'].fillna(' ')
df['description'] = df['tagline'].map(str)+' '+df['overview'].map(str)

In [23]:
df = df.dropna()

In [24]:
df.shape

(4800, 4)

In [25]:
import nltk 
import re
nltk.download('stopwords')
nltk.download('punkt')
stop_words = nltk.corpus.stopwords.words('english')

def normalize_corpus(doc):
  doc = re.sub(r'[^a-zA-Z0-9/s]',' ',doc, re.I | re.A)
  doc = doc.lower()
  doc = doc.strip()
  tokens = nltk.word_tokenize(doc)
  filtered_toks = [token for token in tokens if token not in stop_words]
  doc = " ".join(filtered_toks)
  return doc


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [29]:
norm_corp = np.vectorize(normalize_corpus)
corp = norm_corp(list(df['description']))

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range = (1,2),min_df =2)
tfidf_matrix = tf.fit_transform(corp)
tfidf_matrix.shape

(4800, 20796)

In [31]:
from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,4790,4791,4792,4793,4794,4795,4796,4797,4798,4799
0,1.000000,0.009070,0.000000,0.015911,0.024535,0.018863,0.000000,0.024484,0.000000,0.005971,...,0.008198,0.0,0.020806,0.029501,0.000000,0.000000,0.0,0.005942,0.000000,0.000000
1,0.009070,1.000000,0.011727,0.000000,0.040078,0.000000,0.012674,0.042249,0.032498,0.006880,...,0.009447,0.0,0.004771,0.000000,0.000000,0.012631,0.0,0.022335,0.012902,0.000000
2,0.000000,0.011727,1.000000,0.000000,0.000000,0.000000,0.000000,0.022559,0.014447,0.004508,...,0.040703,0.0,0.000000,0.000000,0.015676,0.000000,0.0,0.011741,0.000000,0.003888
3,0.015911,0.000000,0.000000,1.000000,0.008087,0.003294,0.013174,0.022198,0.023666,0.120670,...,0.000000,0.0,0.009305,0.000000,0.000000,0.000000,0.0,0.026817,0.032627,0.026083
4,0.024535,0.040078,0.000000,0.008087,1.000000,0.000000,0.008659,0.028700,0.000000,0.029824,...,0.013757,0.0,0.000000,0.000000,0.000000,0.010533,0.0,0.022298,0.000000,0.000000


In [40]:
lst_movies = df['title'].values

In [42]:
np.where(lst_movies == 'Interstellar')

(array([95]),)

In [43]:
lst_movies = df['title'].values
def recommendation_movies(title,lst_movies = lst_movies,doc_sim = doc_sim_df):
  #Id of the Movie
  movie_id = np.where(lst_movies == title)[0][0]
  #get the similarity
  mvie_sim = doc_sim.iloc[movie_id].values
  #Top 5 id
  similar_ids = np.argsort(-mvie_sim)[1:6]
  #Get the movie_name maps 
  similar_movies = lst_movies[similar_ids]
  return similar_movies


In [44]:
recommendation_movies('Interstellar')

array(['Gattaca', 'Space Cowboys', 'Space Pirate Captain Harlock',
       'Final Destination 2', 'Starship Troopers'], dtype=object)

In [45]:
recommendation_movies('Minions')

array(['Despicable Me 2', 'Despicable Me', 'Superman', 'Stuart Little 2',
       'Freeway'], dtype=object)

In [72]:
from gensim.models import FastText 
tokenized_corpus = [doc.split() for doc in corp]

#Play with iterations, Window size to get better results - Try with 50 Iterations
ft = FastText(tokenized_corpus,size = 300,min_count = 2,window = 20, workers = 1,sg = 1,iter = 5)

In [73]:
def averaged_wordvector_vectorizer(corpus,model,num_features):
  vocabulary = set(model.wv.index2word)
  def average_word_vectors(words, model,vocabulary,num_features):
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.

    for word in words: 
      if word in vocabulary:
        nwords = nwords+1
        feature_vector = np.add(feature_vector, model.wv[word])
      if nwords: 
        feature_vector = np.divide(feature_vector, nwords)
    return feature_vector
  
  features = [average_word_vectors(tokenized_sentence,model,vocabulary,num_features) for tokenized_sentence in corpus]
  return np.array(features)

In [76]:
doc_vec_ft = averaged_wordvector_vectorizer(tokenized_corpus,ft,300)

In [77]:
from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(doc_vec_ft)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,4790,4791,4792,4793,4794,4795,4796,4797,4798,4799
0,1.000000,0.566251,0.927027,0.954457,0.831282,0.736115,0.798343,0.703983,0.731685,0.935190,...,0.926380,0.480650,0.594093,0.563890,0.962438,0.766132,0.664787,0.686963,0.757386,0.786240
1,0.566251,1.000000,0.789839,0.494551,0.834156,0.872745,0.895895,0.690901,0.851231,0.656232,...,0.773985,0.926380,0.937992,0.956379,0.541815,0.712628,0.887673,0.800993,0.865713,0.826904
2,0.927027,0.789839,1.000000,0.885238,0.917907,0.883452,0.947401,0.766536,0.831460,0.933090,...,0.969575,0.706650,0.814900,0.781689,0.923476,0.890913,0.819260,0.802532,0.900836,0.897050
3,0.954457,0.494551,0.885238,1.000000,0.794961,0.617135,0.786726,0.541760,0.656211,0.865742,...,0.866240,0.432534,0.473595,0.523561,0.936922,0.814508,0.575365,0.610692,0.655890,0.670003
4,0.831282,0.834156,0.917907,0.794961,1.000000,0.903101,0.904222,0.782792,0.920216,0.831646,...,0.954940,0.759167,0.833672,0.858806,0.828249,0.765225,0.894302,0.738346,0.851455,0.864031


In [78]:
lst_movies = df['title'].values
def recommendation_movies(title,lst_movies = lst_movies,doc_sim = doc_sim_df):
  #Id of the Movie
  movie_id = np.where(lst_movies == title)[0][0]
  #get the similarity
  mvie_sim = doc_sim.iloc[movie_id].values
  #Top 5 id
  similar_ids = np.argsort(-mvie_sim)[1:6]
  #Get the movie_name maps 
  similar_movies = lst_movies[similar_ids]
  return similar_movies


In [79]:
recommendation_movies('Interstellar')

array(['White Squall', 'History of the World: Part I', 'Butterfly Girl',
       'The Wind That Shakes the Barley',
       'Underworld: Rise of the Lycans'], dtype=object)